**Description**: Crunch numbers about gaps b/t adjacent submissions in the RAFT
leaderboard to put accuracy differences in context.

**Estimated runtime**: 1 sec.

In [1]:
import io

import polars as pl

copy-pasted from https://huggingface.co/spaces/ought/raft-leaderboard

In [2]:
tsv = r"""Rank	Submitter	Submission Name	Submission Date	Overall	Ade Corpus V2	Banking 77	Neurips Impact Statement Risks	One Stop English	Overruling	Semiconductor Org Types	Systematic Review Inclusion	Tai Safety Research	Terms Of Service	Tweet Eval Hate	Twitter Complaints
1	tedbagel	miles	Nov 18, 2023	0.773	0.813	0.776	0.812	0.7450.944	0.935	0.508	0.722	0.751	0.614	0.887
2	Raldir	AuT-Few (H)	Nov 19, 2022	0.773	0.837	0.647	0.780	0.8470.942	0.917	0.687	0.703	0.728	0.517	0.892
3	AaronLi	yiwise	Jul 08, 2022	0.768	0.856	0.695	0.839	0.698	0.9440.906	0.493	0.737	0.749	0.647	0.883
4	jtmohta	T-Few	May 06, 2022	0.758	0.804	0.695	0.833	0.676	0.9500.915	0.508	0.736	0.750	0.586	0.879
5	Raldir	AuT-Few	Dec 17, 2022	0.747	0.846	0.587	0.898	0.770	0.9630.801	0.620	0.742	0.738	0.350	0.901
6	OE-Heart	yiwise	Jun 13, 2022	0.738	0.793	0.636	0.833	0.6430.948	0.907	0.509	0.693	0.725	0.545	0.886
7	annagon	ada_t	Feb 04, 2023	0.736	0.838	0.639	0.819	0.787	0.9390.853	0.493	0.688	0.712	0.464	0.862
8	ought	Human baseline (crowdsourced)	Aug 27, 2021	0.735	0.830	0.6070.857	0.646	0.917	0.908	0.468	0.609	0.627	0.722	0.897
9	alon-albalak	EXP3-FLAD-3B-Ensemble	Mar 22, 2023	0.735	0.789	0.6780.864	0.533	0.937	0.907	0.493	0.685	0.748	0.584	0.869
10	AaronLi	yiwise	Jun 26, 2022	0.733	0.856	0.464	0.839	0.544	0.9440.906	0.493	0.737	0.749	0.647	0.878
11	alon-albalak	UCB1-FLAD-3B-Ensemble	Mar 22, 2023	0.732	0.796	0.7010.879	0.514	0.940	0.900	0.493	0.689	0.760	0.510	0.871
12	alon-albalak	EXP3-FLAD-3B	Mar 28, 2023	0.729	0.762	0.623	0.8500.618	0.936	0.863	0.493	0.687	0.691	0.633	0.861
13	jogonba2	iatp	Feb 04, 2023	0.724	0.835	0.662	0.826	0.6600.940	0.853	0.493	0.697	0.697	0.444	0.862
14	alon-albalak	EXP3-FLAD-3B	Apr 30, 2023	0.721	0.738	0.542	0.8910.643	0.940	0.828	0.493	0.700	0.709	0.580	0.871
15	moshew	SetFit300	Jul 16, 2022	0.713	0.799	0.632	0.859	0.7600.930	0.769	0.503	0.664	0.604	0.487	0.831
16	JeanneRbs	IAL-602-v2	Aug 19, 2022	0.709	0.814	0.571	0.7960.796	0.915	0.663	0.606	0.693	0.575	0.521	0.850
17	JeanneRbs	IAL-602	Jul 28, 2022	0.706	0.821	0.549	0.776	0.7960.910	0.607	0.606	0.685	0.637	0.521	0.855
18	sxjscience	test-submission11B--2	Sep 23, 2022	0.703	0.823	0.0120.871	0.791	0.947	0.917	0.523	0.728	0.722	0.517	0.878
19	alon-albalak	UCB1-FLAD-3B	Apr 30, 2023	0.701	0.808	0.588	0.8770.421	0.936	0.885	0.493	0.679	0.602	0.545	0.881
20	alon-albalak	UCB1-FLAD-3B	Mar 28, 2023	0.700	0.690	0.590	0.8440.496	0.937	0.880	0.493	0.689	0.712	0.502	0.871
21	Raldir	test-submission-11b-r	Sep 24, 2022	0.699	0.797	0.012	0.8420.793	0.939	0.920	0.493	0.754	0.703	0.542	0.894
22	timoschick	PET	Nov 08, 2021	0.696	0.822	0.593	0.857	0.6460.908	0.816	0.493	0.638	0.576	0.483	0.824
23	seonghyeonye	FLIPPED-Tfew	Nov 20, 2022	0.690	0.755	0.663	0.7940.220	0.938	0.903	0.483	0.736	0.694	0.533	0.871
24	OE-Heart	yiwise	Jun 02, 2022	0.681	0.793	0.012	0.833	0.6430.948	0.907	0.509	0.693	0.725	0.545	0.886
25	moshew	SetFit	Oct 20, 2021	0.669	0.726	0.538	0.872	0.521	0.9070.682	0.493	0.628	0.620	0.532	0.837
26	smangrul	sample_test_peft	Dec 09, 2022	0.660	0.707	0.5360.800	0.319	0.916	0.889	0.493	0.564	0.711	0.464	0.863
27	smangrul	lora-t0-3b	Dec 22, 2022	0.660	0.707	0.536	0.8000.319	0.916	0.889	0.493	0.564	0.711	0.464	0.863
28	APauli	anchor	Dec 23, 2022	0.654	0.777	0.171	0.759	0.711	0.9260.674	0.493	0.672	0.671	0.499	0.845
29	Daromog	NLI_V1	Dec 24, 2022	0.646	0.789	0.619	0.590	0.359	0.9280.757	0.493	0.638	0.624	0.500	0.814
30	APauli	AnchorE	Jan 01, 2023	0.640	0.774	0.067	0.752	0.700	0.8910.728	0.493	0.679	0.616	0.528	0.817
31	APauli	Anchor_300	Apr 07, 2023	0.638	0.774	0.213	0.773	0.5390.896	0.636	0.493	0.683	0.687	0.526	0.798
32	ought	GPT-3	Sep 28, 2021	0.627	0.686	0.299	0.679	0.431	0.9370.769	0.516	0.656	0.574	0.526	0.821
33	HLaci	SetFitBase	Sep 18, 2023	0.624	0.729	0.164	0.659	0.6760.892	0.634	0.493	0.641	0.652	0.488	0.832
34	TomyAndi	PERFECT	Jun 14, 2022	0.618	0.602	0.636	0.755	0.5290.833	0.477	0.509	0.532	0.676	0.507	0.744
35	muelletm	symb-v1	Sep 21, 2022	0.617	0.655	0.629	0.660	0.2950.893	0.459	0.492	0.683	0.664	0.503	0.850
36	Daromog	PET_NLI	Dec 18, 2022	0.609	0.789	0.256	0.590	0.359	0.9280.744	0.493	0.638	0.624	0.490	0.792
37	bugidi	SCARFS	Apr 10, 2022	0.608	0.586	0.542	0.673	0.283	0.8370.710	0.493	0.593	0.689	0.499	0.788
38	HLaci	SetFitCentroid	Oct 22, 2023	0.607	0.702	0.222	0.599	0.4970.906	0.733	0.493	0.615	0.604	0.486	0.824
39	MarioMartone	Surprise Classifier	Feb 22, 2023	0.604	0.541	0.6280.793	0.284	0.884	0.700	0.403	0.615	0.459	0.560	0.781
40	moanik	masf	Oct 28, 2023	0.601	0.702	0.222	0.582	0.555	0.9030.733	0.493	0.513	0.602	0.484	0.824
41	Linuxdex	AG-mb	Feb 25, 2023	0.599	0.623	0.532	0.678	0.3030.821	0.635	0.524	0.596	0.597	0.530	0.753
42	Joocjun	gdragon	May 21, 2023	0.594	0.612	0.063	0.790	0.385	0.9110.845	0.393	0.613	0.527	0.618	0.774
43	yiuli	ftf	Jan 21, 2023	0.593	0.815	0.569	0.527	0.319	0.8620.627	0.408	0.615	0.454	0.499	0.830
44	Eagleking	gcompany	May 21, 2023	0.580	0.588	0.070	0.7250.313	0.881	0.807	0.402	0.641	0.517	0.635	0.807
45	Deyae	AFDA	Jun 24, 2022	0.572	0.637	0.393	0.662	0.166	0.8970.675	0.493	0.626	0.492	0.453	0.802
46	Linuxdex	AG-tt	Mar 20, 2023	0.563	0.481	0.528	0.651	0.3360.908	0.546	0.493	0.639	0.565	0.298	0.743
47	bugidi	RealBoazRoy	Jan 10, 2022	0.561	0.589	0.155	0.593	0.2970.846	0.813	0.493	0.622	0.579	0.517	0.668
48	Linuxdex	AG-mb	Feb 12, 2023	0.555	0.661	0.138	0.588	0.2930.871	0.474	0.493	0.630	0.642	0.519	0.798
49	MarioMartone	SC-zero-shot	Jan 29, 2023	0.553	0.570	0.587	0.5660.330	0.847	0.534	0.376	0.533	0.458	0.615	0.666
50	MarioMartone	Surprise	Feb 18, 2023	0.553	0.570	0.587	0.5660.330	0.847	0.534	0.376	0.533	0.458	0.615	0.666
51	arnaudstiegler	HS-tc	Apr 15, 2022	0.528	0.578	0.067	0.636	0.4350.826	0.367	0.509	0.602	0.591	0.481	0.717
52	muelletm	snlt	Oct 05, 2021	0.528	0.603	0.248	0.585	0.3020.831	0.336	0.492	0.626	0.540	0.449	0.791
53	koaning	does this work	Apr 26, 2023	0.514	0.615	0.176	0.525	0.2890.798	0.332	0.492	0.599	0.614	0.462	0.751
54	ought	AdaBoost	Sep 28, 2021	0.514	0.543	0.023	0.626	0.4750.838	0.455	0.506	0.556	0.560	0.443	0.625
55	Grasshopper12	ccme	May 21, 2023	0.511	0.559	0.064	0.576	0.2950.833	0.819	0.250	0.583	0.308	0.578	0.754
56	MarioMartone	SC zs flan-t5	Mar 03, 2023	0.491	0.565	0.418	0.4220.327	0.746	0.489	0.266	0.508	0.445	0.558	0.652
57	VictorSanh	T0++ zero-shot	Jan 10, 2022	0.488	0.408	0.224	0.6970.321	0.750	0.624	0.493	0.454	0.441	0.377	0.584
58	as980	Fluffy	May 14, 2023	0.486	0.484	0.012	0.572	0.345	0.8740.292	0.383	0.621	0.555	0.460	0.744
59	ought	GPT-Neo	Sep 28, 2021	0.481	0.452	0.149	0.408	0.343	0.6810.406	0.493	0.605	0.565	0.554	0.636
60	Happyjo	go	May 19, 2023	0.476	0.471	0.043	0.540	0.279	0.7840.768	0.202	0.575	0.178	0.605	0.790
61	Happyjo	cdrom	May 21, 2023	0.476	0.471	0.043	0.540	0.279	0.7840.768	0.202	0.575	0.178	0.605	0.790
62	canwenxu	0-shot SC BART	Feb 03, 2023	0.473	0.488	0.489	0.5520.327	0.437	0.532	0.234	0.513	0.460	0.569	0.598
63	canwenxu	0-shot SC	Jan 21, 2023	0.473	0.488	0.489	0.5520.327	0.437	0.532	0.234	0.513	0.460	0.569	0.598
64	ought	GPT-2	Sep 28, 2021	0.458	0.600	0.121	0.561	0.245	0.4980.380	0.492	0.612	0.498	0.311	0.723
65	JanH	Jdist0	May 12, 2023	0.433	0.448	0.012	0.559	0.345	0.5060.292	0.350	0.492	0.417	0.483	0.864
66	cici13306	JdistStudent	May 13, 2023	0.427	0.448	0.012	0.5590.345	0.506	0.292	0.350	0.492	0.417	0.483	0.790
67	seungone	so337	May 21, 2023	0.425	0.493	0.011	0.462	0.3510.492	0.360	0.488	0.493	0.514	0.517	0.493
68	louisdebroglie	ldb1137	May 21, 2023	0.422	0.494	0.010	0.450	0.3500.487	0.377	0.496	0.516	0.516	0.458	0.490
69	doyoungkim	dyk325	May 21, 2023	0.420	0.499	0.009	0.498	0.2840.486	0.370	0.500	0.492	0.494	0.494	0.497
70	CaffreyR	Caffrey	Jul 11, 2022	0.417	0.652	0.012	0.559	0.3450.506	0.292	0.350	0.492	0.417	0.483	0.478
71	lksenel	tz2pzs	Dec 30, 2021	0.417	0.403	0.001	0.353	0.216	0.6960.504	0.514	0.497	0.364	0.366	0.669
72	SkyeDu	PET-albern-base	Nov 02, 2022	0.411	0.448	0.012	0.559	0.3450.506	0.292	0.350	0.492	0.417	0.477	0.628
73	mrcommonsense	mcs1125	May 21, 2023	0.409	0.500	0.012	0.484	0.3370.480	0.367	0.493	0.478	0.479	0.375	0.492
74	koaning	does this work	Apr 21, 2023	0.398	0.448	0.012	0.559	0.3450.506	0.292	0.350	0.492	0.417	0.483	0.478
75	Anon126	WSIAP	May 01, 2022	0.398	0.448	0.012	0.559	0.345	0.5060.292	0.350	0.492	0.417	0.483	0.478
76	VictorSanh	T0++	Jan 10, 2022	0.387	0.560	0.111	0.440	0.3010.347	0.344	0.493	0.415	0.441	0.495	0.309
77	ought	BART MNLI zero-shot	Sep 28, 2021	0.382	0.234	0.332	0.6150.360	0.462	0.644	0.026	0.469	0.122	0.543	0.400
78	jogonba2	zs-ca	Jan 20, 2023	0.348	0.342	0.452	0.542	0.3460.487	0.322	0.073	0.344	0.271	0.303	0.350
79	ought	Plurality-class	Sep 27, 2021	0.331	0.446	0.000	0.353	0.1640.337	0.271	0.493	0.344	0.471	0.366	0.391
80	Daromog	NLI_ZS	Jan 29, 2023	0.296	0.167	0.044	0.415	0.337	0.3370.757	0.108	0.390	0.103	0.309	0.289
81	ought	GPT-3 zero-shot	Sep 28, 2021	0.292	0.163	0.000	0.572	0.3230.378	0.628	0.027	0.362	0.164	0.303	0.290"""

In [3]:
df = pl.read_csv(io.StringIO(tsv), separator="\t").sort("Overall", descending=True)

In [4]:
def acc_diff(x, y):
    return x - y


def odds_ratio(x, y):
    return (x / (1 - x)) / (y / (1 - y))

In [5]:
gaps_acc = pl.Series(
    [acc_diff(df["Overall"][i], df["Overall"][i + 1]) for i in range(len(df) - 1)]
)

gaps_acc.describe()

statistic,value
str,f64
"""count""",80.0
"""null_count""",0.0
"""mean""",0.006013
"""std""",0.007052
"""min""",0.0
"""25%""",0.002
"""50%""",0.003
"""75%""",0.008
"""max""",0.035


In [6]:
gaps_or = pl.Series(
    [odds_ratio(df["Overall"][i], df["Overall"][i + 1]) for i in range(len(df) - 1)]
)

gaps_or.describe()

statistic,value
str,f64
"""count""",80.0
"""null_count""",0.0
"""mean""",1.027227
"""std""",0.032647
"""min""",1.0
"""25%""",1.008158
"""50%""",1.015076
"""75%""",1.035792
"""max""",1.176746
